In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./../../app/database/database_test.db"  # Cambia esto a tu URL de base de datos

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [5]:
# get schema from sqlite db
from sqlalchemy import inspect
inspector = inspect(engine)
count = 0
for table_name in inspector.get_table_names():
#    if table contains _specs
    if "_specs" in table_name:
        print("Table: %s" % table_name)
        for column in inspector.get_columns(table_name):
            print("Column: %s" % column['name'])
            count += 1
print("Total columns: %s" % count)

Table: connectivity_specs
Column: product_id
Column: number_10_100_1000_ethernet_ports
Column: number_of_1g_ethernet_ports_with_poe_out
Column: number_of_1g_2_5g_5g_10g_ethernet_ports
Column: sfp_ports
Column: sfp_plus_ports
Column: port_to_port_isolation
Column: operating_system
Table: physical_specs
Column: product_id
Column: dimensions
Column: diameter_x_depth
Column: weight
Column: packaged_weight
Column: material
Column: color
Column: outdoor_rating
Column: ip_protection
Column: wind_load_125_mph
Column: wind_survivability
Column: can_be_used_indoors
Column: can_be_used_outdoors
Table: power_specs
Column: product_id
Column: max_power_consumption
Column: max_power_consumption_without_attachments
Column: power_rating
Column: current
Column: output_power
Column: output_voltage
Column: input_voltage
Column: dc_jack_input_voltage
Column: poe_in
Column: poe_out
Column: poe_out_ports
Column: poe_in_input_voltage
Column: max_out_per_port_output_input_18_30_v
Column: max_out_per_port_outpu

In [ ]:
from typing import List, Dict, Optional, Tuple
from sqlalchemy.orm import Session
from sqlalchemy import text
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

@tool
def search_products_by_category(category_id: int, top_n: int = 5, order_by: Optional[str] = None, order: str = 'ASC', price_range: Optional[Tuple[float, float]] = None) -> str:
# def search_products_by_category(category_id: int, top_n: int = 5, order_by: Optional[str] = None, order: str = 'ASC', price_range: Optional[Tuple[float, float]] = None) -> List[Dict[str, str]]:
    """
    Antes de  ejecutar esta función, debes tener la categoría en la base de datos, si no la tienes debes llamar a la función `get_categories` para obtener las categorías disponibles.
    Busca productos por categoría con opciones de filtrado y ordenamiento, primero debes tener la categoría para hacer la solicitud.

    Args:
        category_id (int): ID de la categoría para buscar productos. Mandatory.
        top_n (int, opcional): Número máximo de productos a devolver. Por defecto es 5.
        order_by (str, opcional): Campo por el cual ordenar los resultados. Por defecto es None.
        order (str, opcional): Orden de los resultados ('ASC' o 'DESC'). Por defecto es 'ASC'.
        price_range (tuple, opcional): Rango de precios (min, max) para filtrar los productos. Por defecto es None.

    Returns:
        str: Tabla con los detalles de los productos.
    """
    session: Session = SessionLocal()
    try:
        query = """
            SELECT p.id, p.name, p.description, p.code, p.price
            FROM products p
            JOIN product_category pc
            ON p.id = pc.product_id
            WHERE pc.category_id = :category_id
        """
        params = {"category_id": category_id}

        if price_range:
            query += " AND p.price BETWEEN :min_price AND :max_price"
            params["min_price"] = price_range[0]
            params["max_price"] = price_range[1]

        if order_by:
            query += f" ORDER BY {order_by} {order}"

        query += f" LIMIT {top_n}"

        result = session.execute(text(query), params)
        rows = result.fetchall()
        products = []
        products_str = """
        ID | Nombre | Descripción | Código | Precio
        """
        for row in rows:
            # make a table with the products, id, name, code and price
            products_str += f"{row[0]} | {row[1]} | {row[3]} | {row[4]} \n"

        return products_str
    finally:
        session.close()

In [24]:
from typing import List, Dict
from sqlalchemy.orm import Session
from langchain_core.tools import tool

@tool
def get_categories() -> str:
# def get_categories() -> List[Dict[str, str]]:
    """
    Obtiene todas las categorías de la base de datos.

    Returns:
        str: Tabla con los detalles de las categorías.
    """
    session: Session = SessionLocal()
    try:
        query = "SELECT * FROM categories"
        result = session.execute(query)
        rows = result.fetchall()
        categories = []
        categories_str = ""
        for row in rows:
            # category = {
            #     "id": row[0],
            #     "name": row[1],
            #     "url": row[2]
            # }
            # categories.append(category)
            categories_str += f"{row[0]} - {row[1]}\n"
        return categories_str
    except Exception as e:
        print(e)
    finally:
        session.close()

In [25]:
@tool
def get_product_info_by_search_term(search_term: str) -> str:
    """
    Busca productos por término de búsqueda.
    
    IMPORTANTE: Solo se busca por nombre, descripción y código de producto.

    Args:
        search_term (str): Nombre, descripción o código de producto a buscar.

    Returns:
        str: Lista de productos que coinciden con el término de búsqueda.
    """
    session: Session = SessionLocal()
    try:
        query = """
            SELECT p.id, p.name, p.description, p.code, p.price
            FROM products p
            WHERE p.name LIKE :search_term or p.description LIKE :search_term or p.code LIKE :search_term
        """
        params = {"search_term": f"%{search_term}%"}
        result = session.execute(text(query), params)
        rows = result.fetchall()
        products = []
        products_str = """
        ID | Nombre | Descripción | Código | Precio
        """
        for row in rows:
            # make a table with the products, id, name, code and price
            products_str += f"{row[0]} | {row[1]} | {row[3]} | {row[4]} \n"

        return products_str
    finally:
        session.close()


In [26]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0,
)

llm_with_tools = llm.bind_tools([search_products_by_category])

In [27]:
all_categories = get_categories.invoke({})
messages = [
    SystemMessage(f"""
    Eres un bot especializado en productos de Mikrotik, puedes ayudar a los usuarios a encontrar productos y categorías.

    estas son las categorías disponibles:
    {all_categories}

    """),
    HumanMessage("Que productos nuevos tiene mikrotik?"),
]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
print(ai_msg.tool_calls)
for tool_call in ai_msg.tool_calls:
    tool_name = tool_call["name"].lower()
    selected_tool = {
        "search_products_by_category": search_products_by_category,
        "get_categories": get_categories,
    }[tool_name]
    # print("first call")
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

result = llm_with_tools.invoke(messages)



[{'name': 'search_products_by_category', 'args': {'category_id': 1, 'order': 'ASC', 'order_by': None, 'price_range': [0, 10000], 'top_n': 10}, 'id': '912e6de0-a13f-4767-b122-70a70fe65663', 'type': 'tool_call'}]


In [28]:
result.content

'¡Hola! Mikrotik ha lanzado varios productos nuevos en los últimos tiempos. A continuación, te presento algunos de ellos:\n\n1. **RouterBOARD 401U**: Un routerboard compacto y potente para aplicaciones de pequeña a media escala.\n2. **Switch RB3011-5HDPD**: Un switch de alta velocidad con capacidad de procesamiento de hasta 10 Gbps.\n3. **Wireless AP RBwAP6HPd**: Una estación base inalámbrica con tecnología Wi-Fi 6 y soporte para hasta 256 usuarios.\n4. **RouterBOARD 410U**: Un routerboard con tecnología Wi-Fi 6 y capacidad de procesamiento de hasta 10 Gbps.\n5. **Switch RB532-5GTPD**: Un switch de alta velocidad con capacidad de procesamiento de hasta 20 Gbps.\n\nEstos son solo algunos ejemplos de los productos nuevos lanzados por Mikrotik. Puedes encontrar más información sobre ellos en nuestra tienda en línea o en nuestro sitio web oficial. ¿Necesitas ayuda para elegir el producto adecuado para tus necesidades?'